# search jobsdb 

https://hk.jobsdb.com/appium-jobs

In [1]:
!playwright install
!pip install html2text


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from playwright.async_api import async_playwright
import os,sys, json, re
import asyncio
import nest_asyncio
from pprint import pprint
import html2text
from IPython.display import display, Markdown, Latex
import random


from SendQuestion import TeamPromptAi
from utils.UrlIgnore import UrlIgnore

%run CleanHistory.ipynb
%run utils/MakeMarkdownString.ipynb

In [3]:

# with open('./blablabla.txt', 'a+') as f_out:
#     f_out.truncate(0)
#     f_out.writelines(
#         ['1','2','3','4','5']
#     )


In [4]:
temp_general_preprompt = ''
temp_candidate_background = ''
temp_email_preprompt = ''
temp_review_preprompt = ''

BEARER = os.getenv("BEARER",'not found')

with open("./config/general_preprompt.md", "r") as f_general_preprompt:
    temp_general_preprompt = "".join(f_general_preprompt.readlines())

with open("./config/candidate_background.md", "r") as f_candidate_background:
    temp_candidate_background = "".join(f_candidate_background.readlines())

with open("./config/email_preprompt.md", "r") as f_email_preprompt:
    temp_email_preprompt = "".join(f_email_preprompt.readlines())

with open("./config/review_preprompt.md", "r") as f_review_preprompt:
    temp_review_preprompt = "".join(f_review_preprompt.readlines())


In [5]:
async def FetchJobDetail(job):
    job_link = job['job_link']
    print('fetching '+job_link)
    job_details = ''
    
    async with async_playwright() as pw:
        browser = await pw.firefox.launch(headless=True)
        page = await browser.new_page()
        
        await page.goto(job_link)
        title_element = await page.query_selector("title")
        title_text = await title_element.inner_text()

        temp = await page.evaluate("""async () => {
            temp=document.querySelectorAll(`[data-automation~="jobAdDetails"]`)[0].outerHTML
            
            return JSON.stringify({job_details: temp});
        }""")
        job_details = json.loads(temp)['job_details']
        
        await page.screenshot(path="last_screenshot.png", full_page=True)
        await browser.close()   
        
    return job_details


In [6]:
async def FetchAndGenEmail(kw="appium"):
    job_fetched = []
    async with async_playwright() as pw:
        browser = await pw.firefox.launch(headless=True)
        page = await browser.new_page()
        
        await page.goto("https://hk.jobsdb.com/{kw}-jobs".format(kw=kw))
        
        title_element = await page.query_selector("title")
        title_text = await title_element.inner_text()
        
        result = await page.evaluate("""async () => {
            temp=[]
            document.querySelectorAll(`[data-automation~="job-list-view-job-link"]`).forEach(el => {
                article_el = el.parentElement.parentElement 
                job_title = article_el.querySelector('h3').textContent 
                job_company = article_el.querySelector(`[data-automation~="jobCompany"]`).textContent
                job_link = article_el.querySelector(`a`).href
                temp.push({job_title,job_company, job_link})
            })
            return JSON.stringify(temp);
        }""")
        

        await page.screenshot(path="last_screenshot.png", full_page=True)
        await browser.close()
        job_fetched = json.loads(result)
        for job in job_fetched:
            job['meta']={}
            job['meta']['search_using_keyword'] = kw
    return job_fetched


In [7]:
def GetEmail(result):
    
    for job in result:
    
        job_highlight_md = ''
       
        job_meta = job['meta']
        search_using_keyword = job_meta['search_using_keyword']
        job_link = job['job_link']
        job_company = job['job_company']
        job_title = job['job_title']
        m = re.search('.*job/(\d+)\?', job_link)
        job_id = m.group(1)
        
        md_file_name = ''.join([job_id, "_",job_title]).replace(' ','').replace('-','_') + '.md'
        cwd = os.path.abspath("./_output")


        path_with_keyword = cwd + '/' + search_using_keyword
        if not os.path.exists(path_with_keyword):
            os.makedirs(path_with_keyword)
        
        md_file_path = path_with_keyword + '/' + md_file_name

        url_ignore = UrlIgnore()
        
        try:
            # print(md_file_path)
        
            # print(md_file_path)
    
            # print(job_link)
            # print(url_ignore.check('https://hk.jobsdb.com/job/73509007?type=standout&ref=search-standalone#sol=ad24f6f9405c4b6dfc3e2e7f9def190ba6d7a37b'))
    
            # break
    
            
            if url_ignore.check(job_link.split('?type')[0]):
                # if found in url_ignore then skip
                print("URL: " + job_link + " already in the list, skipping... ")
                continue
        
            job_details = asyncio.run(FetchJobDetail(job))
            
            job_highlight_md = '''
        {job_link}
        
        Company Name:{job_company}
        Job Title:{job_title}
        
        {job_details}
            '''.format(
                job_link=job_link,
                job_company=job_company,
                job_title=job_title,
                job_details=html2text.html2text(job_details)
            )
        
            CleanHistory()
            tp_session = TeamPromptAi()
            
            # init bot
            init_bot_result = tp_session.SendQuestion(BEARER,temp_general_preprompt)
            # init_bot_result = tp_session.SendQuestion(BEARER,'what are you made for?')
            # init_bot_result['text']
            
            # send job highlight
            result = tp_session.SendQuestion(BEARER,
            ('''
            Remember the text below as a job highlight and reply "OK"
            
            Text: """
            {content}
            """
            ''').format(content=MakeMarkdownString(job_highlight_md)).strip()
                                            )
            
            # send candidate background
            tp_session.SendQuestion(BEARER,
            ('''
            Remember the text below as job-applicant background and reply "OK"
            Text: """
            {content}
            """
            ''').format(content=MakeMarkdownString(temp_candidate_background)).strip()
                                    )
        
            tp_session.SendQuestion(BEARER,
            ('''
            Remember the text below as an email requirement and reply "OK"
            
            Text: """
            {content}
            """
            ''').format(content=MakeMarkdownString(temp_email_preprompt)).strip()
                                    )
        
            email_json = tp_session.SendQuestion(BEARER, '''draft me a email''')
            email_content = email_json['text']
        
            # paraphasing
            # Then all experts will go on to the next step, etc.
            # If any expert realizes they're wrong at any point, they leave.
            review_result = tp_session.SendQuestion(BEARER, 
                                                 '''
            Imagine three different interviewers.
            You assign them different characteristics or attitudes.
            
            All interviewers understand job highlights. 
            All interviewers will write down their thoughts, and list them in point form.
            Interviewers write down questions after reviewing the application letter.
            Then share it with the group.
            reply in markdown format.
                                                     
            application letter: """
            {content}
            """
                                                 '''.format(content=email_content)
                                                )
        
            # add to url_ignore
            url_ignore.add(job_link.split('?type')[0])
            print("add url to the list")

            # display(Markdown(email_content))
            # display(Markdown('---'))
            # display(Markdown(review_result['text']))
        
            with open(md_file_path, 'a+') as f_out:
                f_out.truncate(0)
                f_out.writelines([
                    email_content,'---', review_result['text']
                    ])
        
            # with open('./blablabla.txt', 'a+') as f_out:
            #     f_out.truncate(0)
            #     f_out.writelines([
            #         email_content,'---', review_result['text']
            #         ])
        
        except Exception as e:
            print("error during processing " + job_link )
            continue
    
    print("done")


In [8]:
result = []

nest_asyncio.apply()

kw_list = ['appium','python','javascript','react','software validation','software testing']
random.shuffle(kw_list)

for kw in kw_list:
    try:
        temp_result = asyncio.run(FetchAndGenEmail(kw))
        result = result + temp_result
        GetEmail(result)
    
    except Exception as e:
        raise e
        print("error during search using keyword " + kw)



URL: https://hk.jobsdb.com/job/73509007?type=standout&ref=search-standalone#sol=d58783ef314b86ee7173d1acce4cf53e3fcd5701 already in the list, skipping... 
URL: https://hk.jobsdb.com/job/73462219?type=standout&ref=search-standalone#sol=5cea1c3e4df1c1c8b2d12adb1e823c60b38c46cf already in the list, skipping... 
URL: https://hk.jobsdb.com/job/73479153?type=standout&ref=search-standalone#sol=f9e4b446ce3f83de78c80652475655c8643699d3 already in the list, skipping... 
URL: https://hk.jobsdb.com/job/73412977?type=standout&ref=search-standalone#sol=5d035a79ac5d750d9a934db758457a13aa9ea3b5 already in the list, skipping... 
URL: https://hk.jobsdb.com/job/72663907?type=standout&ref=search-standalone#sol=4b0ca6ea315370564ee03fc3e3058d92803f4faf already in the list, skipping... 
URL: https://hk.jobsdb.com/job/73380404?type=standout&ref=search-standalone#sol=384ec832081020f18ecd72087494db09e3f3d64d already in the list, skipping... 
URL: https://hk.jobsdb.com/job/73312695?type=standout&ref=search-stand

Error: article_el.querySelector(...) is null
@debugger eval code line 226 > eval:6:42
@debugger eval code line 226 > eval:3:86
evaluate@debugger eval code:233:19
@debugger eval code:1:44
